<h1> VNL Individual Stats 2021-2023</h1>

The below code creates a dataframe with the following columns:
- Player_ID
- Year
- TeamA
- TeamB
- Serve_Points
- Serve_Errors
- Serve_Attempts
- Total_Serves
- Match_Date
- Set_Successes
- Rebounds
- Block_Total_Attempt
- Reception_Successful
- Reception_Errors
- Attempts
- Recep_Total_Attempt
- Dig_Success
- Dig_Errors
- Dig_Attempts
- Dig_Total

<h2> Importing Libraries </h2>

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

<h2> Importing Data </h2>

In [2]:
#Reading in df from Men's VNL Roster notebook
df = pd.read_csv('df_womens_rosters_21_23.csv')

In [3]:
#Making Player_ID Column an integer to paste into the URL
df['Player_ID'] = df['Player_ID'].fillna(0).astype(int)

In [4]:
df.head()

,No.,Player Name,Position,Player_ID,Year,Country_Name,Nationality,Age,Height,Total Points,Average by Match,Attack Points,Efficiency,Attack Avg Points,Block Points,Block Success,Block Avg Points,Serve Points,Serve Success,Serve Avg Points
0,2.0,Van Sas Elise,S,160209,2021,Belgium,Belgium,24.0,188cm,4,0.33,0,0.00%,0.00,3,60.00%,0.25,1,7.69%,0.08
1,3.0,Herbots Britt,OH,152042,2021,Belgium,Belgium,22.0,182cm,337,24.07,314,40.67%,22.43,12,12.63%,0.86,11,5.85%,0.79
2,4.0,Lemmens Nathalie,MB,135880,2021,Belgium,Belgium,27.0,192cm,15,1.07,8,30.77%,0.57,6,12.24%,0.43,1,1.75%,0.07
3,5.0,Guilliams Jodie,OH,156326,2021,Belgium,Belgium,24.0,180cm,86,5.73,73,27.97%,4.87,7,10.00%,0.47,6,3.95%,0.40
4,6.0,Gilson Helena,OH,181274,2021,Belgium,Belgium,22.0,184cm,9,0.69,9,16.36%,0.69,0,0.00%,0.00,0,0.00%,0.00


<h1> Gather Serving Information </h1>

In [5]:
# Initialize the WebDriver for Chrome
def init_webdriver(chromedriver_path):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to find elements by CSS selector and return their texts, excluding empty strings
def find_elements_texts(driver, css_selector):
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    return [element.text for element in elements if element.text.strip()]

# Function to scrape player details from a URL
def scrape_player_details(driver, player_id, year):
    url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}#serve'
    driver.get(url)
    time.sleep(5)  # Adjust as necessary

    # Extract text for each match
    serve_points = find_elements_texts(driver, ".vbw-o-table__cell.serve-points")
    serve_errors = find_elements_texts(driver, ".vbw-o-table__cell.faults")
    serve_attempts = find_elements_texts(driver, ".vbw-o-table__cell.hits")
    total_serves = find_elements_texts(driver,".vbw-o-table__cell.total-attempts")
    match_dates = find_elements_texts(driver, ".vbw-o-table__cell.matchdate")
    teamA = find_elements_texts(driver,"td.vbw-o-table__cell.team.teama .vbw-mu__team__name-link")
    teamB = find_elements_texts(driver,"td.vbw-o-table__cell.team.teamb .vbw-mu__team__name-link")
    
    
    # Get the length of the longest list
    longest_list_length = max(len(serve_points), len(serve_errors), len(serve_attempts), len(match_dates))

    games_data = []
    for i in range(longest_list_length):
        game_data = {
            'Player_ID': player_id,
            'Year': year,
            'TeamA': teamA[i] if i < len(teamA) else None,
            'TeamB': teamB[i] if i < len(teamB) else None,
            'Serve_Points': serve_points[i] if i < len(serve_points) else None,
            'Serve_Errors': serve_errors[i] if i < len(serve_errors) else None,
            'Serve_Attempts': serve_attempts[i] if i < len(serve_attempts) else None,
            'Total_Serves': total_serves[i] if i < len(total_serves) else None,
            'Match_Date': match_dates[i] if i < len(match_dates) else None,
        }
        games_data.append(game_data)
    
    return games_data

# Specify the path to your ChromeDriver
chromedriver_path = '/Users/zakirpasha/Downloads/chromedriver-mac-x64/chromedriver'
driver = init_webdriver(chromedriver_path)

# Assuming `df` is your DataFrame containing player IDs and positions
all_games_data = []
#for key, df in dataframes.items():
for index, row in df.iterrows():
    if row['Position'] == 'COACH':
        continue  # Skip rows where the position is COACH
    year = row['Year']  # Assuming 'Year' is directly available in each row
    player_id = row['Player_ID']
    games_data = scrape_player_details(driver, player_id, year)  # Use the driver instance here
    all_games_data.extend(games_data)  # Extend, not append, to add each game's data individually


# Close the WebDriver
driver.quit()

# Convert the collected data into a DataFrame
final_df_serve = pd.DataFrame(all_games_data)
final_df_serve.reset_index(drop=True, inplace=True)

final_df_serve.head()

,Player_ID,Year,TeamA,TeamB,Serve_Points,Serve_Errors,Serve_Attempts,Total_Serves,Match_Date
0,160209,2021,Belgium,Canada,1,1,2,4,20/06/2021
1,160209,2021,Belgium,Italy,0,0,0,0,19/06/2021
2,160209,2021,Thailand,Belgium,0,0,0,0,18/06/2021
3,160209,2021,Belgium,Japan,0,0,0,0,14/06/2021
4,160209,2021,Türkiye,Belgium,0,0,0,0,13/06/2021


<h1> Gather Setting Information </h1>

In [6]:
# Initialize the WebDriver for Chrome
def init_webdriver(chromedriver_path):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to find elements by CSS selector and return their texts, excluding empty strings
def find_elements_texts(driver, css_selector):
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    return [element.text for element in elements if element.text.strip()]

# Function to scrape player details from a URL
def scrape_player_details(driver, player_id, year):
    url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}#set'
    driver.get(url)
    time.sleep(5)  # Adjust as necessary

    # Extract text for each match
    Successful = find_elements_texts(driver, ".vbw-o-table__cell.running-sets")
    Errors = find_elements_texts(driver, ".vbw-o-table__cell.faults")
    Attempts = find_elements_texts(driver, ".vbw-o-table__cell.still-sets")
    set_total_attempt = find_elements_texts(driver,".vbw-o-table__cell.total-attempts")
    match_dates = find_elements_texts(driver, ".vbw-o-table__cell.matchdate")
    teamA = find_elements_texts(driver,"td.vbw-o-table__cell.team.teama .vbw-mu__team__name-link")
    teamB = find_elements_texts(driver,"td.vbw-o-table__cell.team.teamb .vbw-mu__team__name-link")
    
    
    # Get the length of the longest list
    longest_list_length = max(len(Successful), len(Errors), len(Attempts), len(set_total_attempt))

    games_data = []
    for i in range(longest_list_length):
        game_data = {
            'Player_ID': player_id,
            'Year': year,
            'TeamA': teamA[i] if i < len(teamA) else None,
            'TeamB': teamB[i] if i < len(teamB) else None,
            'Set_Successes': Successful[i] if i < len(Successful) else None,
            'Set_Errors': Errors[i] if i < len(Errors) else None,
            'Set_Attempts': Attempts[i] if i < len(Attempts) else None,
            'Total_Sets':set_total_attempt[i] if i < len(set_total_attempt) else None,
            'Match_Date': match_dates[i] if i < len(match_dates) else None,
        }
        games_data.append(game_data)
    
    return games_data

# Specify the path to your ChromeDriver
chromedriver_path = '/Users/zakirpasha/Downloads/chromedriver-mac-x64/chromedriver'
driver = init_webdriver(chromedriver_path)

# Assuming `df` is your DataFrame containing player IDs and positions
all_games_data = []
#for key, df in dataframes.items():
for index, row in df.iterrows():
    if row['Position'] == 'COACH':
        continue  # Skip rows where the position is COACH
    year = row['Year']  # Assuming 'Year' is directly available in each row
    player_id = row['Player_ID']
    games_data = scrape_player_details(driver, player_id, year)  # Use the driver instance here
    all_games_data.extend(games_data)  # Extend, not append, to add each game's data individually


# Close the WebDriver
driver.quit()

# Convert the collected data into a DataFrame
final_df_set = pd.DataFrame(all_games_data)
final_df_set.reset_index(drop=True, inplace=True)

final_df_set.head()

,Player_ID,Year,TeamA,TeamB,Set_Successes,Set_Errors,Set_Attempts,Total_Sets,Match_Date
0,160209,2021,Belgium,Canada,2,0,10,12,20/06/2021
1,160209,2021,Belgium,Italy,0,0,0,0,19/06/2021
2,160209,2021,Thailand,Belgium,0,0,2,2,18/06/2021
3,160209,2021,Belgium,Japan,0,0,0,0,14/06/2021
4,160209,2021,Türkiye,Belgium,0,0,0,0,13/06/2021


<h1> Gathering Attacking Information </h1>

In [7]:
# Initialize the WebDriver for Chrome
def init_webdriver(chromedriver_path):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to find elements by CSS selector and return their texts, excluding empty strings
def find_elements_texts(driver, css_selector):
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    return [element.text for element in elements if element.text.strip()]

# Function to scrape player details from a URL
def scrape_player_details(driver, player_id, year):
    url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}#attack'
    driver.get(url)
    time.sleep(5)  # Adjust as necessary

    # Extract text for each match
    Successful = find_elements_texts(driver, ".vbw-o-table__cell.attacks")
    Errors = find_elements_texts(driver, ".vbw-o-table__cell.faults")
    Attempts = find_elements_texts(driver, ".vbw-o-table__cell.shots")
    attack_total_attempt = find_elements_texts(driver,".vbw-o-table__cell.total-attempts")
    match_dates = find_elements_texts(driver, ".vbw-o-table__cell.matchdate")
    teamA = find_elements_texts(driver,"td.vbw-o-table__cell.team.teama .vbw-mu__team__name-link")
    teamB = find_elements_texts(driver,"td.vbw-o-table__cell.team.teamb .vbw-mu__team__name-link")
    
    
    # Get the length of the longest list
    longest_list_length = max(len(Successful), len(Errors), len(Attempts), len(attack_total_attempt))
    num_games = len(match_dates)
    
    games_data = []
    for i in range(longest_list_length):
        game_data = {
            'Player_ID': player_id,
            'Year': year,
            'TeamA': teamA[i] if i < len(teamA) else None,
            'TeamB': teamB[i] if i < len(teamB) else None,
            'Attack_Success': Successful[i] if i < len(Successful) else None,
            'Attack Errors': Errors[i] if i < len(Errors) else None,
            'Attack_Attempts': Attempts[i] if i < len(Attempts) else None,
            'Total_Attacks':attack_total_attempt[i] if i < len(attack_total_attempt) else None,
            'Match_Date': match_dates[i] if i < len(match_dates) else None,
        }
        games_data.append(game_data)
    
    return games_data

# Specify the path to your ChromeDriver
chromedriver_path = '/Users/zakirpasha/Downloads/chromedriver-mac-x64/chromedriver'
driver = init_webdriver(chromedriver_path)

# Assuming `df` is your DataFrame containing player IDs and positions
all_games_data = []
#for key, df in dataframes.items():
for index, row in df.iterrows():
    if row['Position'] == 'COACH':
        continue  # Skip rows where the position is COACH
    year = row['Year']  # Assuming 'Year' is directly available in each row
    player_id = row['Player_ID']
    games_data = scrape_player_details(driver, player_id, year)  # Use the driver instance here
    all_games_data.extend(games_data)  # Extend, not append, to add each game's data individually


# Close the WebDriver
driver.quit()

# Convert the collected data into a DataFrame
final_df_attack = pd.DataFrame(all_games_data)
final_df_attack.reset_index(drop=True, inplace=True)

final_df_attack.head()

,Player_ID,Year,TeamA,TeamB,Attack_Success,Attack Errors,Attack_Attempts,Total_Attacks,Match_Date
0,160209,2021,Belgium,Canada,0,0,0,0,20/06/2021
1,160209,2021,Belgium,Italy,0,0,0,0,19/06/2021
2,160209,2021,Thailand,Belgium,0,0,0,0,18/06/2021
3,160209,2021,Belgium,Japan,0,0,0,0,14/06/2021
4,160209,2021,Türkiye,Belgium,0,0,0,0,13/06/2021


<h1> Gathering Blocking Information </h1>

In [8]:
# Initialize the WebDriver for Chrome
def init_webdriver(chromedriver_path):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to find elements by CSS selector and return their texts, excluding empty strings
def find_elements_texts(driver, css_selector):
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    return [element.text for element in elements if element.text.strip()]

# Function to scrape player details from a URL
def scrape_player_details(driver, player_id, year):
    url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}#block'
    driver.get(url)
    time.sleep(5)  # Adjust as necessary

    # Extract text for each match
    Successful = find_elements_texts(driver, ".vbw-o-table__cell.stuff-blocks")
    Errors = find_elements_texts(driver, ".vbw-o-table__cell.faults")
    Rebounds = find_elements_texts(driver, ".vbw-o-table__cell.rebounds")
    block_total_attempt = find_elements_texts(driver,".vbw-o-table__cell.total-attempts")
    match_dates = find_elements_texts(driver, ".vbw-o-table__cell.matchdate")
    teamA = find_elements_texts(driver,"td.vbw-o-table__cell.team.teama .vbw-mu__team__name-link")
    teamB = find_elements_texts(driver,"td.vbw-o-table__cell.team.teamb .vbw-mu__team__name-link")
    
    
    # Get the length of the longest list
    longest_list_length = max(len(Successful), len(Errors), len(Rebounds), len(block_total_attempt))

    games_data = []
    for i in range(longest_list_length):
        game_data = {
            'Player_ID': player_id,
            'Year': year,
            'TeamA': teamA[i] if i < len(teamA) else None,
            'TeamB': teamB[i] if i < len(teamB) else None,
            'Block_Successful': Successful[i] if i < len(Successful) else None,
            'Block_Errors': Errors[i] if i < len(Errors) else None,
            'Block_Rebounds': Rebounds[i] if i < len(Rebounds) else None,
            'Total_Blocks': block_total_attempt[i] if i < len(block_total_attempt) else None,
            'Match_Date': match_dates[i] if i < len(match_dates) else None,
        }
        games_data.append(game_data)
    
    return games_data

# Specify the path to your ChromeDriver
chromedriver_path = '/Users/zakirpasha/Downloads/chromedriver-mac-x64/chromedriver'
driver = init_webdriver(chromedriver_path)

# Assuming `df` is your DataFrame containing player IDs and positions
all_games_data = []
#for key, df in dataframes.items():
for index, row in df.iterrows():
    if row['Position'] == 'COACH':
        continue  # Skip rows where the position is COACH
    year = row['Year']  # Assuming 'Year' is directly available in each row
    player_id = row['Player_ID']
    games_data = scrape_player_details(driver, player_id, year)  # Use the driver instance here
    all_games_data.extend(games_data)  # Extend, not append, to add each game's data individually


# Close the WebDriver
driver.quit()

# Convert the collected data into a DataFrame
final_df_block = pd.DataFrame(all_games_data)
final_df_block.reset_index(drop=True, inplace=True)

final_df_block.head()

,Player_ID,Year,TeamA,TeamB,Block_Successful,Block_Errors,Block_Rebounds,Total_Blocks,Match_Date
0,160209,2021,Belgium,Canada,1,0,1,2,20/06/2021
1,160209,2021,Belgium,Italy,0,0,0,0,19/06/2021
2,160209,2021,Thailand,Belgium,0,0,0,0,18/06/2021
3,160209,2021,Belgium,Japan,0,0,0,0,14/06/2021
4,160209,2021,Türkiye,Belgium,0,0,0,0,13/06/2021


<h1> Gathering Reception Information </h1>

In [9]:
# Initialize the WebDriver for Chrome
def init_webdriver(chromedriver_path):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to find elements by CSS selector and return their texts, excluding empty strings
def find_elements_texts(driver, css_selector):
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    return [element.text for element in elements if element.text.strip()]

# Function to scrape player details from a URL
def scrape_player_details(driver, player_id, year):
    url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}#reception'
    driver.get(url)
    time.sleep(5)  # Adjust as necessary

    # Extract text for each match
    Successful = find_elements_texts(driver, ".vbw-o-table__cell.excellents")
    Errors = find_elements_texts(driver, ".vbw-o-table__cell.faults")
    Attempts = find_elements_texts(driver, ".vbw-o-table__cell.serve-receptions")
    recep_total_attempt = find_elements_texts(driver,".vbw-o-table__cell.total-attempts")
    match_dates = find_elements_texts(driver, ".vbw-o-table__cell.matchdate")
    teamA = find_elements_texts(driver,"td.vbw-o-table__cell.team.teama .vbw-mu__team__name-link")
    teamB = find_elements_texts(driver,"td.vbw-o-table__cell.team.teamb .vbw-mu__team__name-link")
    
    
    # Get the length of the longest list
    longest_list_length = max(len(Successful), len(Errors), len(Attempts), len(recep_total_attempt))

    games_data = []
    for i in range(longest_list_length):
        game_data = {
            'Player_ID': player_id,
            'Year': year,
            'TeamA': teamA[i] if i < len(teamA) else None,
            'TeamB': teamB[i] if i < len(teamB) else None,
            'Reception_Successful': Successful[i] if i < len(Successful) else None,
            'Reception_Errors': Errors[i] if i < len(Errors) else None,
            'Reception_Attemps': Attempts[i] if i < len(Attempts) else None,
            'Total_Receptions': recep_total_attempt[i] if i < len(recep_total_attempt) else None,
            'Match_Date': match_dates[i] if i < len(match_dates) else None,
        }
        games_data.append(game_data)
    
    return games_data

# Specify the path to your ChromeDriver
chromedriver_path = '/Users/zakirpasha/Downloads/chromedriver-mac-x64/chromedriver'
driver = init_webdriver(chromedriver_path)

# Assuming `df` is your DataFrame containing player IDs and positions
all_games_data = []
#for key, df in dataframes.items():
for index, row in df.iterrows():
    if row['Position'] == 'COACH':
        continue  # Skip rows where the position is COACH
    year = row['Year']  # Assuming 'Year' is directly available in each row
    player_id = row['Player_ID']
    games_data = scrape_player_details(driver, player_id, year)  # Use the driver instance here
    all_games_data.extend(games_data)  # Extend, not append, to add each game's data individually


# Close the WebDriver
driver.quit()

# Convert the collected data into a DataFrame
final_df_recep = pd.DataFrame(all_games_data)
final_df_recep.reset_index(drop=True, inplace=True)

final_df_recep.head()

,Player_ID,Year,TeamA,TeamB,Reception_Successful,Reception_Errors,Reception_Attemps,Total_Receptions,Match_Date
0,160209,2021,Belgium,Canada,0,0,0,0,20/06/2021
1,160209,2021,Belgium,Italy,0,0,0,0,19/06/2021
2,160209,2021,Thailand,Belgium,0,0,0,0,18/06/2021
3,160209,2021,Belgium,Japan,0,0,0,0,14/06/2021
4,160209,2021,Türkiye,Belgium,0,0,0,0,13/06/2021


<h1> Gathering Dig Info </h1>

In [10]:
# Initialize the WebDriver for Chrome
def init_webdriver(chromedriver_path):
    service = Service(executable_path=chromedriver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to find elements by CSS selector and return their texts, excluding empty strings
def find_elements_texts(driver, css_selector):
    elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
    return [element.text for element in elements if element.text.strip()]

# Function to scrape player details from a URL
def scrape_player_details(driver, player_id, year):
    url = f'https://en.volleyballworld.com/volleyball/competitions/volleyball-nations-league/{year}/players/{player_id}#dig'
    driver.get(url)
    time.sleep(5)  # Adjust as necessary

    # Extract text for each match
    Successful = find_elements_texts(driver, ".vbw-o-table__cell.great-save")
    Errors = find_elements_texts(driver, ".vbw-o-table__cell.faults")
    Attempts = find_elements_texts(driver, ".vbw-o-table__cell.receptions")
    dig_total_attempt = find_elements_texts(driver,".vbw-o-table__cell.total-attempts")
    match_dates = find_elements_texts(driver, ".vbw-o-table__cell.matchdate")
    teamA = find_elements_texts(driver,"td.vbw-o-table__cell.team.teama .vbw-mu__team__name-link")
    teamB = find_elements_texts(driver,"td.vbw-o-table__cell.team.teamb .vbw-mu__team__name-link")
    
    
    # Get the length of the longest list
    longest_list_length = max(len(Successful), len(Errors), len(Attempts), len(dig_total_attempt))

    games_data = []
    for i in range(longest_list_length):
        game_data = {
            'Player_ID': player_id,
            'Year': year,
            'TeamA': teamA[i] if i < len(teamA) else None,
            'TeamB': teamB[i] if i < len(teamB) else None,
            'Dig_Success': Successful[i] if i < len(Successful) else None,
            'Dig_Errors': Errors[i] if i < len(Errors) else None,
            'Dig_Attempts': Attempts[i] if i < len(Attempts) else None,
            'Total_Digs': dig_total_attempt[i] if i < len(dig_total_attempt) else None,
            'Match_Date': match_dates[i] if i < len(match_dates) else None,
        }
        games_data.append(game_data)
    
    return games_data

# Specify the path to your ChromeDriver
chromedriver_path = '/Users/zakirpasha/Downloads/chromedriver-mac-x64/chromedriver'
driver = init_webdriver(chromedriver_path)

# Assuming `df` is your DataFrame containing player IDs and positions
all_games_data = []
#for key, df in dataframes.items():
for index, row in df.iterrows():
    if row['Position'] == 'COACH':
        continue  # Skip rows where the position is COACH
    year = row['Year']  # Assuming 'Year' is directly available in each row
    player_id = row['Player_ID']
    games_data = scrape_player_details(driver, player_id, year)  # Use the driver instance here
    all_games_data.extend(games_data)  # Extend, not append, to add each game's data individually


# Close the WebDriver
driver.quit()

# Convert the collected data into a DataFrame
final_df_dig = pd.DataFrame(all_games_data)
final_df_dig.reset_index(drop=True, inplace=True)

final_df_dig.head()

,Player_ID,Year,TeamA,TeamB,Dig_Success,Dig_Errors,Dig_Attempts,Total_Digs,Match_Date
0,160209,2021,Belgium,Canada,1,0,1,2,20/06/2021
1,160209,2021,Belgium,Italy,0,0,0,0,19/06/2021
2,160209,2021,Thailand,Belgium,0,0,0,0,18/06/2021
3,160209,2021,Belgium,Japan,0,0,0,0,14/06/2021
4,160209,2021,Türkiye,Belgium,0,0,0,0,13/06/2021


<h1> Joining Overall + Metric dataframes Together </h1>

In [11]:
df_joined1 = pd.merge(final_df_serve, final_df_set, on=['Player_ID', 'Year', 'TeamA', 'TeamB','Match_Date'], how='left')

In [12]:
df_joined2 = pd.merge(df_joined1, final_df_attack, on=['Player_ID', 'Year', 'TeamA', 'TeamB','Match_Date'], how='left')

In [13]:
df_joined3 = pd.merge(df_joined2, final_df_block, on=['Player_ID', 'Year', 'TeamA', 'TeamB','Match_Date'], how='left')

In [14]:
df_joined4 = pd.merge(df_joined3, final_df_recep, on=['Player_ID', 'Year', 'TeamA', 'TeamB','Match_Date'], how='left')

In [15]:
df_womens_indv_21_23 = pd.merge(df_joined4, final_df_dig, on=['Player_ID', 'Year', 'TeamA', 'TeamB','Match_Date'], how='left')

In [16]:
df_womens_indv_21_23.to_csv('df_womens_indv_21_23.csv')